In [39]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.datasets import ImageFolder
import json
from torchvision.transforms.functional import to_pil_image
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import os
import cv2
from torchvision.transforms import functional as Func

In [40]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [41]:
device = "cpu"

In [67]:
num_epochs = 5
batch_size = 4
learning_rate = 0.01

In [68]:
annotation_root = '/home/maria/Desktop/Master degree/2 course/computer vision/final project/Bottle-Defect-Detection-4'
json_file = "_annotations.coco.json"

In [69]:
def json_to_dict(file, annotation_root=annotation_root):

    file_path = f"{annotation_root}/{file}"

    try:
        with open(file_path, 'r') as json_file:
            data = json.load(json_file)
        result_list = [
            {
                "id": image["id"],
                "category_id": image["category_id"],
                "xmin": image["bbox"][0],
                "xmax": image["bbox"][0]+ image["bbox"][1],
                "ymin": image['bbox'][2],
                "ymax": image['bbox'][2]+ image['bbox'][3],
                "target": image['bbox']
            }
            for image in data.get("annotations", [])
        ]

        return result_list

    except json.JSONDecodeError as e:
        print(f"Error decoding JSON in file {file}: {e}")
        return None

result_dict = json_to_dict(json_file, annotation_root)
result_dict

[{'id': 0,
  'category_id': 5,
  'xmin': 253,
  'xmax': 296,
  'ymin': 251.84,
  'ymax': 758.89,
  'target': [253, 43, 251.84, 507.05]},
 {'id': 1,
  'category_id': 3,
  'xmin': 253,
  'xmax': 508,
  'ymin': 248.84,
  'ymax': 356.7,
  'target': [253, 255, 248.84, 107.86]},
 {'id': 2,
  'category_id': 1,
  'xmin': 310,
  'xmax': 350,
  'ymin': 130.35,
  'ymax': 206.64999999999998,
  'target': [310, 40, 130.35, 76.3]},
 {'id': 3,
  'category_id': 4,
  'xmin': 527,
  'xmax': 736,
  'ymin': 182.3,
  'ymax': 314.43,
  'target': [527, 209, 182.3, 132.13]},
 {'id': 4,
  'category_id': 3,
  'xmin': 367,
  'xmax': 1101,
  'ymin': 406.29,
  'ymax': 709.94,
  'target': [367, 734, 406.29, 303.65]},
 {'id': 5,
  'category_id': 2,
  'xmin': 347,
  'xmax': 550,
  'ymin': 463.62,
  'ymax': 1871.08,
  'target': [347, 203, 463.62, 1407.46]},
 {'id': 6,
  'category_id': 3,
  'xmin': 188,
  'xmax': 398,
  'ymin': 222.86,
  'ymax': 302.96000000000004,
  'target': [188, 210, 222.86, 80.1]},
 {'id': 7,
  'ca

In [70]:
def convert_to_dict(list_of_dicts):
    return {item['id']: item['category_id'] for item in list_of_dicts}

In [71]:
data_labels=convert_to_dict(result_dict)

In [72]:
data_labels

{0: 5,
 1: 3,
 2: 1,
 3: 4,
 4: 3,
 5: 2,
 6: 3,
 7: 1,
 8: 5,
 9: 5,
 10: 5,
 11: 1,
 12: 1,
 13: 3,
 14: 3,
 15: 1,
 16: 1,
 17: 5,
 18: 5,
 19: 5,
 20: 3,
 21: 1,
 22: 4,
 23: 3,
 24: 2,
 25: 3,
 26: 4,
 27: 5,
 28: 1,
 29: 3,
 30: 5,
 31: 1,
 32: 4,
 33: 3,
 34: 4,
 35: 3,
 36: 2,
 37: 2,
 38: 1,
 39: 5,
 40: 4,
 41: 3,
 42: 5,
 43: 1,
 44: 3,
 45: 5,
 46: 1,
 47: 3,
 48: 3,
 49: 4,
 50: 2,
 51: 4,
 52: 3,
 53: 2,
 54: 5,
 55: 1,
 56: 3,
 57: 4,
 58: 3,
 59: 5,
 60: 4,
 61: 3,
 62: 2,
 63: 4,
 64: 3,
 65: 2,
 66: 3,
 67: 4,
 68: 2,
 69: 3,
 70: 4,
 71: 4,
 72: 4,
 73: 3,
 74: 2,
 75: 3,
 76: 1,
 77: 5,
 78: 5,
 79: 4,
 80: 3,
 81: 1,
 82: 5,
 83: 1,
 84: 1,
 85: 3,
 86: 3,
 87: 5,
 88: 5,
 89: 4,
 90: 3,
 91: 2,
 92: 3,
 93: 4,
 94: 2,
 95: 3,
 96: 4,
 97: 2,
 98: 3,
 99: 4,
 100: 2,
 101: 3,
 102: 4,
 103: 2,
 104: 2,
 105: 4,
 106: 3,
 107: 3,
 108: 1,
 109: 5,
 110: 1,
 111: 5,
 112: 3,
 113: 4,
 114: 2,
 115: 5,
 116: 1,
 117: 1,
 118: 5,
 119: 5,
 120: 1,
 121: 1,
 122: 5,
 12

In [73]:
def convert_to_dict4(list_of_dicts):
    return {item['id']: item['target'] for item in list_of_dicts}

In [74]:
data_rectangles=convert_to_dict3(result_dict)

In [75]:
data_rectangles

{0: [253, 43, 251.84, 507.05],
 1: [253, 255, 248.84, 107.86],
 2: [310, 40, 130.35, 76.3],
 3: [527, 209, 182.3, 132.13],
 4: [367, 734, 406.29, 303.65],
 5: [347, 203, 463.62, 1407.46],
 6: [188, 210, 222.86, 80.1],
 7: [243, 67, 119.87, 45.56],
 8: [190, 59, 223.36, 464.8],
 9: [830, 284, 108.66, 253.84],
 10: [1027, 269, 121.77, 244.41],
 11: [877, 285, 38.67, 28.41],
 12: [1090, 273, 38.35, 21.26],
 13: [101, 185, 214.13, 175.09],
 14: [309, 155, 215.59, 159.88],
 15: [163, 2, 73.89, 46.79],
 16: [360, 10, 80.35, 38.79],
 17: [303, 2, 233.84, 638.5],
 18: [93, 0, 218.88, 639.75],
 19: [309, 49, 220.34, 558.05],
 20: [309, 262, 215.37, 135.6],
 21: [363, 58, 122.85, 58.3],
 22: [554, 6, 230.29, 154.74],
 23: [464, 574, 392.28, 359.67],
 24: [441, 0, 458.27, 1547.33],
 25: [672, 420, 68.41, 50.68],
 26: [689, 343, 44.41, 34.42],
 27: [667, 340, 79.55, 169.28],
 28: [245, 65, 116.37, 43.81],
 29: [188, 190, 216.86, 97.88],
 30: [193, 58, 222.86, 464.55],
 31: [357, 137, 432.29, 1460.

In [76]:
def json_to_dict2(file, annotation_root=annotation_root):

    file_path = f"{annotation_root}/{file}"

    try:
        with open(file_path, 'r') as json_file:
            data = json.load(json_file)
        result_list = [
            {
                "id": image["id"],
                "filename": image['file_name']
            }
            for image in data.get("images", [])
        ]

        return result_list

    except json.JSONDecodeError as e:
        print(f"Error decoding JSON in file {file}: {e}")
        return None
file_dict = json_to_dict2(json_file, annotation_root)
file_dict

[{'id': 0, 'filename': '100003_jpg.rf.f52e6e997c347d3ed2feb567faf4a5ef.jpg'},
 {'id': 1,
  'filename': 'vlcsnap-2023-06-22-12h50m14s593_png.rf.f60b1b74cadda4dc86abc4a44f90d28e.jpg'},
 {'id': 2, 'filename': '100002_jpg.rf.fb8afbc2239e3d3e61640072a459e43d.jpg'},
 {'id': 3,
  'filename': 'vlcsnap-2023-07-10-21h49m49s278_png.rf.fb7b409f245aabc96ca3b12eec8235d2.jpg'},
 {'id': 4, 'filename': '000093_jpg.rf.fa09ef470ad7af3b5ce09a7b8f8f85e2.jpg'},
 {'id': 5, 'filename': '000038_jpg.rf.fcc8cb9486e90e38cad366e569e38cb8.jpg'},
 {'id': 6,
  'filename': 'vlcsnap-2023-06-22-12h12m10s291_png.rf.fce99f4d596a45202f311666f40a67f5.jpg'},
 {'id': 7,
  'filename': 'vlcsnap-2023-07-10-21h47m41s099_png.rf.ff974319ae63fb8a980e6bb1f9566d12.jpg'},
 {'id': 8, 'filename': '100002_jpg.rf.f752ec4eeb39a8f02e6e4185aa91455d.jpg'},
 {'id': 9,
  'filename': 'vlcsnap-2023-06-22-12h10m55s425_png.rf.fea2103a8d6057a652454f7b00b836da.jpg'},
 {'id': 10,
  'filename': 'vlcsnap-2023-06-22-12h11m36s193_png.rf.f667d337076359d8256

In [77]:
def convert_to_dict2(list_of_dicts):
    return {item['id']: item['filename'] for item in list_of_dicts}

In [78]:
data_original_name = convert_to_dict2(file_dict)
data_original_name

{0: '100003_jpg.rf.f52e6e997c347d3ed2feb567faf4a5ef.jpg',
 1: 'vlcsnap-2023-06-22-12h50m14s593_png.rf.f60b1b74cadda4dc86abc4a44f90d28e.jpg',
 2: '100002_jpg.rf.fb8afbc2239e3d3e61640072a459e43d.jpg',
 3: 'vlcsnap-2023-07-10-21h49m49s278_png.rf.fb7b409f245aabc96ca3b12eec8235d2.jpg',
 4: '000093_jpg.rf.fa09ef470ad7af3b5ce09a7b8f8f85e2.jpg',
 5: '000038_jpg.rf.fcc8cb9486e90e38cad366e569e38cb8.jpg',
 6: 'vlcsnap-2023-06-22-12h12m10s291_png.rf.fce99f4d596a45202f311666f40a67f5.jpg',
 7: 'vlcsnap-2023-07-10-21h47m41s099_png.rf.ff974319ae63fb8a980e6bb1f9566d12.jpg',
 8: '100002_jpg.rf.f752ec4eeb39a8f02e6e4185aa91455d.jpg',
 9: 'vlcsnap-2023-06-22-12h10m55s425_png.rf.fea2103a8d6057a652454f7b00b836da.jpg',
 10: 'vlcsnap-2023-06-22-12h11m36s193_png.rf.f667d337076359d8256f6d4fcc7bd3b6.jpg',
 11: '6a00d83451b96069e200e553b8b8168834-800wi_jpg.rf.f979e3e3b666c8db39727c962b0f59cb.jpg',
 12: 'vlcsnap-2023-07-10-21h47m46s050_png.rf.0eff067c1711bdde22204555c9771226.jpg',
 13: '100008_jpg.rf.049703296aa291

In [79]:
def convert_to_dict3(list_of_dicts):
    return {item['filename']: item['id'] for item in list_of_dicts}

data_original_name_2 = convert_to_dict3(file_dict)
data_original_name_2

{'100003_jpg.rf.f52e6e997c347d3ed2feb567faf4a5ef.jpg': 0,
 'vlcsnap-2023-06-22-12h50m14s593_png.rf.f60b1b74cadda4dc86abc4a44f90d28e.jpg': 1,
 '100002_jpg.rf.fb8afbc2239e3d3e61640072a459e43d.jpg': 2,
 'vlcsnap-2023-07-10-21h49m49s278_png.rf.fb7b409f245aabc96ca3b12eec8235d2.jpg': 3,
 '000093_jpg.rf.fa09ef470ad7af3b5ce09a7b8f8f85e2.jpg': 4,
 '000038_jpg.rf.fcc8cb9486e90e38cad366e569e38cb8.jpg': 5,
 'vlcsnap-2023-06-22-12h12m10s291_png.rf.fce99f4d596a45202f311666f40a67f5.jpg': 6,
 'vlcsnap-2023-07-10-21h47m41s099_png.rf.ff974319ae63fb8a980e6bb1f9566d12.jpg': 7,
 '100002_jpg.rf.f752ec4eeb39a8f02e6e4185aa91455d.jpg': 8,
 'vlcsnap-2023-06-22-12h10m55s425_png.rf.fea2103a8d6057a652454f7b00b836da.jpg': 9,
 'vlcsnap-2023-06-22-12h11m36s193_png.rf.f667d337076359d8256f6d4fcc7bd3b6.jpg': 10,
 '6a00d83451b96069e200e553b8b8168834-800wi_jpg.rf.f979e3e3b666c8db39727c962b0f59cb.jpg': 11,
 'vlcsnap-2023-07-10-21h47m46s050_png.rf.0eff067c1711bdde22204555c9771226.jpg': 12,
 '100008_jpg.rf.049703296aa29136d4

In [80]:
original_label = {}
for key,value in data_labels.items():
    if key<262:
        original_label[data_original_name[key]]=value

In [81]:
original_label

{'100003_jpg.rf.f52e6e997c347d3ed2feb567faf4a5ef.jpg': 5,
 'vlcsnap-2023-06-22-12h50m14s593_png.rf.f60b1b74cadda4dc86abc4a44f90d28e.jpg': 3,
 '100002_jpg.rf.fb8afbc2239e3d3e61640072a459e43d.jpg': 1,
 'vlcsnap-2023-07-10-21h49m49s278_png.rf.fb7b409f245aabc96ca3b12eec8235d2.jpg': 4,
 '000093_jpg.rf.fa09ef470ad7af3b5ce09a7b8f8f85e2.jpg': 3,
 '000038_jpg.rf.fcc8cb9486e90e38cad366e569e38cb8.jpg': 2,
 'vlcsnap-2023-06-22-12h12m10s291_png.rf.fce99f4d596a45202f311666f40a67f5.jpg': 3,
 'vlcsnap-2023-07-10-21h47m41s099_png.rf.ff974319ae63fb8a980e6bb1f9566d12.jpg': 1,
 '100002_jpg.rf.f752ec4eeb39a8f02e6e4185aa91455d.jpg': 5,
 'vlcsnap-2023-06-22-12h10m55s425_png.rf.fea2103a8d6057a652454f7b00b836da.jpg': 5,
 'vlcsnap-2023-06-22-12h11m36s193_png.rf.f667d337076359d8256f6d4fcc7bd3b6.jpg': 5,
 '6a00d83451b96069e200e553b8b8168834-800wi_jpg.rf.f979e3e3b666c8db39727c962b0f59cb.jpg': 1,
 'vlcsnap-2023-07-10-21h47m46s050_png.rf.0eff067c1711bdde22204555c9771226.jpg': 1,
 '100008_jpg.rf.049703296aa29136d4805

In [82]:
transform = transforms.Compose(
    [transforms.Resize((400,400)),
                                transforms.ToTensor(),
                               transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
    ]
)

In [83]:
class BottlesDataset(Dataset):
    def __init__(self, root_dir, annotations_file=original_label, transform=None,id_name = data_original_name_2, rectangle = data_rectangles):
        self.image_folder = ImageFolder(root_dir, transform)
        self.classes = list(set(list(annotations_file.values())))
        self.annot = annotations_file
        self.id_name = id_name
        self.rectangle = rectangle
        

    
    def __len__(self):
        return len(self.image_folder)
    
    def __getitem__(self, index):
        image, _ = self.image_folder[index]

         
        file_path = self.image_folder.imgs[index][0]
    
        
        file_name = os.path.basename(file_path)
    
        image_id = self.id_name[file_name]
        label = self.annot[file_name] - 1
        bboxes = self.rectangle[image_id]
        targets = {'labels':label, "bboxes":bboxes}
        return image,  targets


In [84]:
root_dir = '/home/maria/Desktop/Master degree/2 course/computer vision/final project/Bottle-Defect-Detection-4'

In [85]:
bottles = BottlesDataset(root_dir = root_dir,transform = transform)

In [86]:
bottles.classes

[1, 2, 3, 4, 5]

In [88]:
ImageFolder(root_dir,transform)

Dataset ImageFolder
    Number of datapoints: 262
    Root location: C:\Users\Admin\OneDrive\Desktop\Maria_Hovakimyan\Master degree\2 course\Computer Vision\final project\Bottle-Defect-Detection-4
    StandardTransform
Transform: Compose(
               Resize(size=(400, 400), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [89]:
train_size = int(0.8 * len(bottles))
test_size = len(bottles) - train_size
train_dataset, test_dataset = random_split(bottles,[train_size,test_size])
train_loader = DataLoader(train_dataset,batch_size = batch_size, shuffle = True)
test_loader = DataLoader(test_dataset,batch_size = batch_size,shuffle = False)

In [90]:
class ConvNet(nn.Module):
    def __init__(self, num_classes=5):
        super(ConvNet, self).__init__()
        self.num_classes = num_classes
        
        self.conv1 = nn.Conv2d(3, 6, kernel_size=5, stride=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1)
        self.fc1 = nn.Linear(16 * 97 * 97, 256)
        self.fc2 = nn.Linear(256, 64)
        
        
        
        self.fc3 = nn.Linear(64, self.num_classes + 4)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 97 * 97)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        
        class_scores = x[:, :self.num_classes]
        bbox_coords = x[:, self.num_classes:]   
        return class_scores, bbox_coords

 

In [93]:
model = ConvNet()
classification_criterion = nn.CrossEntropyLoss()
regression_criterion = nn.SmoothL1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
n_total_steps = len(train_loader)

for epoch in range(num_epochs):
    for i, (images, targets) in enumerate(train_loader):
        images = images
        class_labels = targets['labels'] 
        bbox_labels = torch.stack(targets['bboxes'])

        
        class_scores, bbox_coords = model(images)

        
        classification_loss = classification_criterion(class_scores, class_labels)

        
        regression_loss = regression_criterion(bbox_coords, bbox_labels)

        
        loss = classification_loss + regression_loss

        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 2 == 0:
            print(f"Epoch[{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}")

print("Finished Training")
PATH = 'cnn.pth'
torch.save(model.state_dict(), PATH)


Epoch[1/5], Step [2/53], Loss: 383.8018
Epoch[1/5], Step [4/53], Loss: 366.3780
Epoch[1/5], Step [6/53], Loss: 273.6558
Epoch[1/5], Step [8/53], Loss: 326.4274
Epoch[1/5], Step [10/53], Loss: 106.7336
Epoch[1/5], Step [12/53], Loss: 224.4038
Epoch[1/5], Step [14/53], Loss: 299.8241
Epoch[1/5], Step [16/53], Loss: 161.2217
Epoch[1/5], Step [18/53], Loss: 395.0103
Epoch[1/5], Step [20/53], Loss: 155.4763
Epoch[1/5], Step [22/53], Loss: 146.3455
Epoch[1/5], Step [24/53], Loss: 186.4166
Epoch[1/5], Step [26/53], Loss: 288.2585
Epoch[1/5], Step [28/53], Loss: 300.9372
Epoch[1/5], Step [30/53], Loss: 265.1889
Epoch[1/5], Step [32/53], Loss: 185.2084
Epoch[1/5], Step [34/53], Loss: 244.3293
Epoch[1/5], Step [36/53], Loss: 138.3419
Epoch[1/5], Step [38/53], Loss: 246.4274
Epoch[1/5], Step [40/53], Loss: 190.5542
Epoch[1/5], Step [42/53], Loss: 346.1706
Epoch[1/5], Step [44/53], Loss: 369.1743
Epoch[1/5], Step [46/53], Loss: 320.2005
Epoch[1/5], Step [48/53], Loss: 186.7640
Epoch[1/5], Step [50

c:\Users\Admin\anaconda3\envs\concept_drift\lib\site-packages\torch\nn\modules\loss.py:933: UserWarning: Using a target size (torch.Size([4, 1])) that is different to the input size (torch.Size([1, 4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.smooth_l1_loss(input, target, reduction=self.reduction, beta=self.beta)


Epoch[2/5], Step [2/53], Loss: 197.4353
Epoch[2/5], Step [4/53], Loss: 310.6173
Epoch[2/5], Step [6/53], Loss: 142.5915
Epoch[2/5], Step [8/53], Loss: 235.4332
Epoch[2/5], Step [10/53], Loss: 223.6853
Epoch[2/5], Step [12/53], Loss: 186.5640
Epoch[2/5], Step [14/53], Loss: 274.2781
Epoch[2/5], Step [16/53], Loss: 176.0985
Epoch[2/5], Step [18/53], Loss: 207.5671
Epoch[2/5], Step [20/53], Loss: 208.3055
Epoch[2/5], Step [22/53], Loss: 248.8762
Epoch[2/5], Step [24/53], Loss: 273.6969
Epoch[2/5], Step [26/53], Loss: 164.2359
Epoch[2/5], Step [28/53], Loss: 197.2979
Epoch[2/5], Step [30/53], Loss: 165.7716
Epoch[2/5], Step [32/53], Loss: 173.7702
Epoch[2/5], Step [34/53], Loss: 204.2881
Epoch[2/5], Step [36/53], Loss: 164.8768
Epoch[2/5], Step [38/53], Loss: 239.5665
Epoch[2/5], Step [40/53], Loss: 179.8226
Epoch[2/5], Step [42/53], Loss: 166.2291
Epoch[2/5], Step [44/53], Loss: 251.3844
Epoch[2/5], Step [46/53], Loss: 212.8866
Epoch[2/5], Step [48/53], Loss: 184.8784
Epoch[2/5], Step [50

In [108]:
model.eval()

ConvNet(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=150544, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=9, bias=True)
)

In [109]:
cap = cv2.VideoCapture('/home/maria/Desktop/Master degree/2 course/computer vision/Noy video.mp4')
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

In [115]:
out = cv2.VideoWriter('output_video_1.mp4', cv2.VideoWriter_fourcc(*'MP4V'), 30, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    
    
    if ret:
        output_frame = frame.copy()
        
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        
        pil_image = Image.fromarray(frame_rgb)
        
        
        input_tensor = transform(pil_image)
        
        
        input_batch = input_tensor.unsqueeze(0)
        
        
        with torch.no_grad():
            prediction = model(input_batch)
        
        for pred in prediction:
            if len(pred.tolist()[0]) == 4:
                continue
            else:
                _,x, y, width, height = pred.tolist()[0]
            
            x, y, width, height = int(x * frame_width), int(y * frame_height), int(width * frame_width), int(height * frame_height)
            
            
            cv2.rectangle(output_frame, (x, y), (x + width, y + height), (255, 0, 0), 2)
            
            
        
        
        processed_pil_image = transforms.ToPILImage()(input_tensor.cpu())
        
        
        processed_frame_rgb = np.array(processed_pil_image)
        
        
        output_frame = cv2.cvtColor(processed_frame_rgb, cv2.COLOR_RGB2BGR)
        
        
        output_frame = cv2.resize(output_frame, (frame_width, frame_height))
        
        
        out.write(output_frame)
    
    else:
        break


cap.release()
out.release()
cv2.destroyAllWindows()